# PyBullet Humanoid with Amazon SageMaker

---
## Introduction

PyBullet is an [open source](https://pybullet.org/wordpress/) physics simulator that is commonly used to train RL policies for simulated robotic systems.  PyBullety provides 3D visualization of physical systems with multiple joints in contact with each other and their environment.

In this notebook we install PyBullet into the SageMaker RL container (see the Dockerfile), and send training jobs of RL algorithms of the RLlib library.

In [ ]:
env = "HumanoidBulletEnv-v0"

## Imports

To get started, we'll import the Python libraries we need, set up the environment with a few prerequisites for permissions and configurations.

In [ ]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
import numpy as np
from IPython.display import HTML
import time
from time import gmtime, strftime

sys.path.append("common")
from common.misc import (
    get_execution_role,
    wait_for_s3_object
)
from common.docker_utils import (
    build_and_push_docker_image
)
from sagemaker.rl import (
    RLEstimator,
    RLToolkit,
    RLFramework
)

IMPORTANT: to continue, setup the AWS credentials follwing https://docs.aws.amazon.com/sdk-for-java/v1/developer-guide/setup-credentials.html

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = #
os.environ['AWS_SECRET_ACCESS_KEY'] = #
os.environ['AWS_REGION'] = #

### Setup S3 bucket

Set up the linkage and authentication to the S3 bucket that you want to use for checkpoint and the metadata. 

In [ ]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()
s3_output_path = "s3://{}/".format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

### Define Variables 

We define variables such as the job prefix for the training jobs

In [ ]:
job_name_prefix = "rl-humanoid-bullet"

### Configure instance type

There are multiple instance types to run RL training jobs in SageMaker. See https://aws.amazon.com/sagemaker/pricing/?nc1=h_ls.

In [ ]:
instance_type = "ml.c5.9xlarge"

### Create an IAM role

Either get the execution role when running from a SageMaker notebook instance `role = sagemaker.get_execution_role()` or, when running from local notebook instance, use utils method `role = get_execution_role()` to create an execution role.

In [ ]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

### Install docker for `local` mode

In order to work in `local` mode, you need to have docker installed. When running from you local machine, please make sure that you have docker and docker-compose (for local CPU machines) and nvidia-docker (for local GPU machines) installed. Alternatively, when running from a SageMaker notebook instance, you can simply run the following script to install dependenceis.

Note, you can only run a single local notebook at one time.

In [ ]:
# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./common/setup.sh

## Build docker container

We must build a custom docker container with Ray, TensorFlow and PyBullet installed.  This takes care of everything:

1. Fetching base container image
2. Installing PyBullet and its dependencies
3. Uploading the new container image to ECR

This step can take a long time.


In [ ]:
%%time

cpu_or_gpu = "gpu" if instance_type.startswith("ml.p") else "cpu"
repository_short_name = "sagemaker-bullet-ray-%s" % cpu_or_gpu
docker_build_args = {
    "CPU_OR_GPU": cpu_or_gpu,
    "AWS_REGION": boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print("Using ECR image %s" % custom_image_name)

## Training

The training code in the /src directory. 

In [ ]:
!pygmentize src/train-humanoid-ppo.py

## Train the RL model using the Python SDK Script mode

When using SageMaker for training, you can select a GPU or CPU instance. The RLEstimator is used for training RL jobs. The follwoing steps must be made:

1. Specify the source directory where the environment, presets and training code is uploaded.
2. Specify the entry point as the training code 
3. Specify the choice of RL toolkit and framework. This automatically resolves to the ECR path for the RL Container. 
4. Define the training parameters such as the instance count, job name, S3 path for output and job name. 
5. Specify the hyperparameters for the RL agent algorithm. The RLCOACH_PRESET or the RLRAY_PRESET can be used to specify the RL agent algorithm you want to use. 
6. Define the metrics definitions that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks. 

In [ ]:
%%time

ALG = 'ppo'

metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)
int_regex = '[0-9]+'
float_regex = "[-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?"  # noqa: W605, E501
metric_definitions.append( {"Name": "timesteps_total", "Regex": "timesteps_total: (%s)" % int_regex} )
metric_definitions.append( {"Name": "time_this_iter_s", "Regex": "time_this_iter_s: (%s)" % float_regex} )

custom_image_name = "565870655207.dkr.ecr.us-west-2.amazonaws.com/sagemaker-bullet-ray-cpu:latest"



estimator = RLEstimator(
    entry_point="train-%s.py" % f"humanoid-{ALG}",
    source_dir="src",
    dependencies=["common/sagemaker_rl"],
    image_uri=custom_image_name,
    role=role,
    instance_type=instance_type,
    instance_count=1,
    output_path=s3_output_path,
    base_job_name=job_name_prefix+'-'+ALG,
    metric_definitions=metric_definitions,
    hyperparameters={
    },
)

estimator.fit(wait=local_mode)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

## Visualization

RL training can take a long time.  So while it's running there are a variety of ways we can track progress of the running training job.  Some intermediate output gets saved to S3 during training, so we'll set up to capture that.

In [ ]:
print("Job name: {}".format(job_name))

s3_url = "s3://{}/{}".format(s3_bucket, job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Intermediate folder path: {}".format(intermediate_url))

tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

### Fetch videos of training 
Videos of certain rollouts get written to S3 during training.  Here we fetch the last 10 videos from S3, and render the last one.

In [ ]:
recent_videos = wait_for_s3_object(
    s3_bucket,
    intermediate_folder_key,
    tmp_dir,
    fetch_only=(lambda obj: obj.key.endswith(".mp4") and obj.size > 0),
    limit=10,
    training_job_name=job_name,
)

In [ ]:
last_video = sorted(recent_videos)[-1]  # Pick which video to watch
os.system("mkdir -p ./src/tmp_render/ && cp {} ./src/tmp_render/last_video.mp4".format(last_video))
HTML('<video src="./src/tmp_render/last_video.mp4" controls autoplay></video>')

### Plot metrics for training job
We can see the reward metric of the training as it's running, using algorithm metrics that are recorded in CloudWatch metrics.  We can plot this to see the performance of the model over time.

In [ ]:
%matplotlib inline
from sagemaker.analytics import TrainingJobAnalytics

if not local_mode:
    df = TrainingJobAnalytics(job_name, ["episode_reward_mean"]).dataframe()
    num_metrics = len(df)
    if num_metrics == 0:
        print("No algorithm metrics found in CloudWatch")
    else:
        plt = df.plot(x="timestamp", y="value", figsize=(12, 5), legend=True, style="b-")
        plt.set_ylabel("Mean reward per episode")
        plt.set_xlabel("Training time (s)")
else:
    print("Can't plot metrics in local mode.")

### Monitor training progress
You can repeatedly run the visualization cells to get the latest videos or see the latest metrics as the training job proceeds.